In [ ]:
import os
import time
import shutil
import subprocess

# Step 0: Print header for script execution
print("\n🎯  sync.ipynb is executing ...")

# Step 1: Try importing Colab-specific module
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Step 2: Set Constants
PROJECT_NAME = "cifar-susume"
BASE_PATH = "/content"
MOUNT_PATH = os.path.join(BASE_PATH, "drive/MyDrive")
SRC_PATH = os.path.join(MOUNT_PATH, "src")
REPO_PATH = os.path.join(SRC_PATH, PROJECT_NAME)
REPO_URL = f"https://github.com/saeedkhademidoroh/{PROJECT_NAME}.git"

# Step 3: Set Flags
FORCE_MOUNT = False
FORCE_CLONE = False
FORCE_SKIP = False

# Step 4: Set Timing Controls
PROCESS_DELAY = 5
PROCESS_TIMEOUT = 25

# Step 5: Mount Google Drive
print("\n🔗  Mounting Google Drive on Colab ...")
if IN_COLAB:
    try:
        drive.mount("/content/drive", force_remount=FORCE_MOUNT)
    except Exception as e:
        raise RuntimeError(
            f"\n\n❌  Error from sync.ipynb during Google Drive mount!\n→ {e}\n\n"
        )
print(f"\n⏳   Waiting {PROCESS_DELAY} seconds to finish mounting ...")
time.sleep(PROCESS_DELAY)

for _ in range(PROCESS_TIMEOUT):
    if os.path.exists(MOUNT_PATH):
        break
    time.sleep(1)
else:
    raise TimeoutError(
        f"\n\n❌  TimeoutError from sync.ipynb during mount phase:\n"
        f"Drive mount did not complete in {PROCESS_TIMEOUT} seconds\n→ Path: '{MOUNT_PATH}'\n\n"
    )

print("\n✅  Google Drive has been mounted on Colab.")

# Step 6: If FORCE_CLONE — delete SRC_PATH
if os.path.exists(SRC_PATH):
    if FORCE_CLONE:
        print("\n🗑️  Deleting source directory ...")
        shutil.rmtree(SRC_PATH)
        os.chdir(BASE_PATH)
        print(f"\n⏳   Waiting {PROCESS_DELAY} seconds to finish deleting ...")
        time.sleep(PROCESS_DELAY)
        for _ in range(PROCESS_TIMEOUT):
            if not os.path.exists(SRC_PATH):
                break
            time.sleep(1)
        else:
            raise TimeoutError(
                f"\n\n❌  TimeoutError from sync.ipynb during deletion phase:\n"
                f"Directory deletion did not complete in {PROCESS_TIMEOUT} seconds\n→ Path: '{SRC_PATH}'\n\n"
            )
        print("\n✅  Source directory has been deleted.")
    else:
        print("\n🧪  FORCE_CLONE is disabled — source directory preserved.")
else:
    print("\n✅  Source directory was already deleted.")

# Step 7: If FORCE_CLONE — create SRC_PATH
if FORCE_CLONE:
    print("\n📁  Creating source directory ...")
    os.makedirs(SRC_PATH, exist_ok=True)
    print(f"\n⏳   Waiting {PROCESS_DELAY} seconds to finish creating ...")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        if os.path.exists(SRC_PATH):
            break
        time.sleep(1)
    else:
        raise TimeoutError(
            f"\n\n❌  TimeoutError from sync.ipynb during creation phase:\n"
            f"Directory creation did not complete in {PROCESS_TIMEOUT} seconds\n→ Path: '{SRC_PATH}'\n\n"
        )
    print("\n✅  Source directory has been created.")
else:
    print("\n🧪  FORCE_CLONE is disabled — skipping directory creation.")

# Step 8: Prepare repository
print("\n📥  Preparing repository on Colab ...")

if not FORCE_CLONE and os.path.exists(REPO_PATH):
    print("\n🔄  Pulling repository from GitHub ...")
    modification_time = os.path.getmtime(os.path.join(REPO_PATH, ".git"))
    subprocess.run(["git", "-C", REPO_PATH, "fetch", "origin"], check=True)
    subprocess.run(["git", "-C", REPO_PATH, "reset", "--hard", "origin/main"], check=True)
    print(f"\n⏳   Waiting {PROCESS_DELAY} seconds to finish pulling ...")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        current_mtime = os.path.getmtime(os.path.join(REPO_PATH, ".git"))
        if current_mtime != modification_time:
            break
        time.sleep(1)
    else:
        raise TimeoutError(
            f"\n\n❌  TimeoutError from sync.ipynb during pull phase:\n"
            f"Repository pull did not complete in {PROCESS_TIMEOUT} seconds\n→ Path: '{REPO_PATH}'\n\n"
        )
    print("\n✅  Repository has been pulled.")
else:
    print(f"\n🌱  Cloning repository from GitHub:\n{REPO_URL} → {REPO_PATH}")
    try:
        subprocess.run(["git", "clone", REPO_URL, REPO_PATH], check=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError(
            f"\n\n❌  Error from sync.ipynb during git clone!\n→ {e}\n\n"
        )

    print(f"\n⏳   Waiting {PROCESS_DELAY} seconds to finish cloning ...")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        if os.path.isdir(REPO_PATH) and os.listdir(REPO_PATH):
            break
        time.sleep(1)
    else:
        raise TimeoutError(
            f"\n\n❌  TimeoutError from sync.ipynb during clone phase:\n"
            f"Repository clone did not complete in {PROCESS_TIMEOUT} seconds\n→ Path: '{REPO_PATH}'\n\n"
        )
    print("\n✅  Repository has been cloned.")

# Step 9: Change directory and run main.py
print(f"\n📂  Changing working directory ...\n→ Target path: {REPO_PATH}")
if os.path.exists(REPO_PATH):
    os.chdir(REPO_PATH)
    print(f"\n⏳   Waiting {PROCESS_DELAY} seconds to finish changing ...")
    time.sleep(PROCESS_DELAY)
    print("\n✅  Directory has been changed.")

    if not FORCE_SKIP:
        print(f"\n🚀  Running main.py as a subprocess:\n[{REPO_PATH}/main.py]\n")
        try:
            process = subprocess.Popen(
                ["python", "-u", "main.py"],
                cwd=REPO_PATH,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True
            )
            for line in process.stdout:
                print(line, end="")
            process.wait()
        except Exception as e:
            raise RuntimeError(
                f"\n\n❌  Error from sync.ipynb during main.py execution!\n→ {e}\n\n"
            )
    else:
        print("\n🧪  FORCE_SKIP is enabled — main.py execution skipped.")
else:
    raise FileNotFoundError(
        f"\n\n❌  Changing directory timed out — REPO_PATH does not exist!\n→ '{REPO_PATH}'\n\n"
    )
